# Структуры

Структуры (`struct`) это **значимые** типы. 

**Особенность:**  
Память под них выделяется in-place, т.е. либо на стеке (для локальных переменных и параметров), либо прямо в памяти объекта (для полей). 

В противоположность, объекты ссылочных типов (`class`/`interface`/`delegate`) размещаются в куче, а в стеке (или внутри других объектов) располагаются ссылки на эти объекты.

Вытекающие:
- Не управляются garbage collector'ом.
- Время жизни ограничено областью видимости (scope).
- При передаче в методы экземпляры структур **копируются**.
- Значение null неприменимо. Инициализируются немного иначе.
- **Не наследуются!** Иначе сломался бы upcast из-за возможного отличия в размере (ссылки всегда одного размера).

## 1. Память выделяется прямо "на месте"

In [1]:
struct Struct
{
    public string Value { get; set; }
}

In [2]:
class MyClass
{
    // Память выделяется на всю структуру прямо внутри объекта MyClass.
    private Struct value;
    
    // Память внутри объекта выделяется только на ссылку!
    private object obj;
    
    public void Method()
    {
        Struct localValue; // Память выделилась на стеке
    }
}

## 2.  Время жизни ограничено scope'ом

In [3]:
void MyMethod()
{

    Struct @struct = new Struct { Value = "Hello" };
    
    object obj = new object();
    
} // В этот момент @struct "умирает", а obj может всё ещё существовать в памяти!

## 3. Передача экземпляров структур в методы

In [4]:
class Class
{
    public string Value { get; set; }
}

// Копируем ссылку: можем обращаться к полям исходного объекта.
void ChangeValue(Class obj)
{
    obj.Value = "New value!";
}

Class @class = new Class{ Value = "Hello" };
ChangeValue(@class);
@class.Value

New value!

In [5]:
// Копируем весь экземпляр: нет доступа к исходному объекту.
void ChangeValue(Struct obj)
{
    obj.Value = "New value!";
}

Struct @struct = new Struct{ Value = "Hello" };
ChangeValue(@struct);
@struct.Value

Hello

Если очень хочется, то можно через ref

In [6]:
// Копируем весь инстанс: нет доступа к исходному объекту.
void ChangeValue(ref Struct obj)
{
    obj.Value = "New value!";
}

Struct @struct = new Struct{ Value = "Hello" };
ChangeValue(ref @struct);
@struct.Value

New value!

## 4. "Конструктор без параметров"

In [7]:
struct Rational
{
    public long Numerator { get; }
    public long Denominator { get; } // = 1;
    
    // public Rational(){}
    
    public Rational(long numerator, long denominator)
    {
        Numerator = numerator;
        Denominator = denominator;
    }
    
    public override string ToString() => $"{Numerator}/{Denominator}";
}

In [8]:
object obj;
obj

In [9]:
Rational value;
value.ToString()

0/0

In [10]:
// Вызывать пустой конструктор 10000 раз?
Rational[] values = new Rational[10000];

Отсылаю к [ответу Джона Скита на SO:](https://stackoverflow.com/a/333840/15753334)

> The parameterless constructor isn't created by the compiler. Value types don't have to have constructors as far as the CLR is concerned - although it turns out it can if you write it in IL. When you write "new Guid()" in C# that emits different IL to what you get if you call a normal constructor.

И ещё одна цитата из [другого вопроса](https://stackoverflow.com/questions/203695/does-using-new-on-a-struct-allocate-it-on-the-heap-or-stack), посвящённого "конструкторам без параметров" в структурах

> ... according to C#, all value types have a parameterless constructor. According to the CLI spec, *no* value types have parameterless constructors. (Fetch the constructors of a value type with reflection some time - you won't find a parameterless one.)

**!!!"Позиция партии" в этом вопросе отличается!!!**
<img src="image1.png" style="width: 500px;">

### Резюмируя:
- В структурах нельзя определять конструктор без параметров. Пустой конструктор не генерируется при компиляции: его просто нет. **Но в ваших презентациях сказано иное - учитывайте это на тесте.** 
- При создании экземпляров структур без явного вызова конструктора с параметрами под структуру просто выделяется память и заполняется нулями.
- Как следствие, недопустима inline инициализация полей и свойств.

## 5. Упаковка (boxing)

Иерархия наследования всех структур: STRUCT -> System.ValueType -> System.Object

Значит, можем приводить экземпляры структур к типу Object. Так как структура - значимый тип, а object - ссылочный, требуется особый механизм приведения, который называется **упаковка (boxing)**. Экземпляр структуры "оборачивается" в ссылочный object и помещается в кучу. Это достаточно затратный процесс, поэтому считается плохой практикой.

In [11]:
struct Struct
{
    public string Value { get; set; }
}

public void ChangeValue(Object obj)
{
    // Обратный процесс - распаковка.
    // Значение копируется в локальную переменную.
    Struct s = (Struct)obj;
    s.Value = "New message";
}

Struct s = new Struct { Value = "Message" };
ChangeValue(s); // Упаковка
s.Value

Message

In [12]:
int five = 5;
// Это тоже упаковка!
ValueType valueType = five;

## 6. Наследование

Понимание того, что структуры не наследуются приводит к довольно широким выводам:

- Модификаторы sealed, abstract, virtual, protected запрещены, т.к. не имеют смысла;
- Структура не может быть статической (не имеет смысла + в IL коде `static class` преобразуется в `abstract sealed class`);

In [13]:
sealed struct Sealed {}

static struct Struct
{
    protected int protectedField;

    public abstract void AbstractMethod();
    
    public virtual void VirtualMethod() {}
}

Unhandled Exception: (1,15): error CS0106: Модификатор "sealed" недопустим для этого элемента.
(3,15): error CS0106: Модификатор "static" недопустим для этого элемента.
(5,19): error CS0666: '"Struct.protectedField": новый защищенный член объявлен в структуре.
(7,26): error CS0106: Модификатор "abstract" недопустим для этого элемента.
(9,25): error CS0106: Модификатор "virtual" недопустим для этого элемента.

**И снова неправда в презентациях, можно переопределять методы object (напр. ToString)**

<img src="image2.png" style="width: 500px;">

## 7. Реализация интерфейсов

Структуры могут реализовывать интерфейсы. 

Однако т.к. интерфейсы **всегда являются ссылочными типами**, при создании интерфейсной ссылки на инстанс структуры, этот инстанс всегда упаковывается. Что может довольно сильно ударить по производительности, если это не учитывать.

In [14]:
interface IntValueKeeper
{
    int Value { get; set; }
}

struct Struct : IntValueKeeper
{
    public int Value { get; set; }
}

In [15]:
void ChangeValue(IntValueKeeper obj, int newValue)
{
    obj.Value = newValue;
}

Struct @struct = new Struct { Value = 24 };

// Тут инстанс структуры копируется и упаковывается, что довольно плохо.
IntValueKeeper boxedStruct = @struct; 

// Передали как ссылочный тип
ChangeValue(boxedStruct, 42);

// boxedStruct - ссылочный объект, поэтому значение поменялось!
boxedStruct.Value

42

In [16]:
// Изначальная структура никак не поменялась
@struct.Value

24

In [17]:
// И опять - сравнение ссылок выдаёт false
(IComparable<int>)42 == (IComparable<int>)42

False

## 8. Сравнение структур

У структур не определен оператор сравнения ==, однако можно сравнивать экземпляры структур с помощью .Equals() - причём по значению, а не по ссылке.

In [18]:
struct Point
{
    public double X { get; set; }
    public double Y { get; set; }
}

In [19]:
Point p1 = new Point { X = 42, Y = 24 };
Point p2 = new Point { X = 42, Y = 24 };

In [20]:
p1 == p2

Unhandled Exception: (1,1): error CS0019: Оператор "==" невозможно применить к операнду типа "Point" и "Point".

In [21]:
p1.Equals(p2)

True

## 9. ref, readonly 

**Модификатор ref запрещает экземплярам структуры временно перейти в управляемую кучу (запрещает boxing).**

Это достигается ограничениями на этапе компиляции.

https://docs.microsoft.com/ru-ru/dotnet/csharp/language-reference/builtin-types/struct#ref-struct

In [22]:
public ref struct RefStruct // : IntValueKeeper
{
    public int Value { get; set; }
}

In [23]:
RefStruct[] refStructs = new RefStruct[10];

Unhandled Exception: (1,1): error CS0611: Элементы массива не могут иметь тип "RefStruct".
(1,30): error CS0611: Элементы массива не могут иметь тип "RefStruct".

In [24]:
class MyClass
{
    RefStruct refStructField;
}

Unhandled Exception: (3,5): error CS8345: Поле или автоматически реализуемое свойство не может быть типа "RefStruct", если это не член экземпляра ссылочной структуры.

In [25]:
object obj = (object)new RefStruct();

Unhandled Exception: (1,14): error CS0030: Не удается преобразовать тип "RefStruct" в "object"

In [26]:
List<RefStruct> list;

Unhandled Exception: (1,17): error CS0306: Тип "RefStruct" не может использоваться в качестве аргумента типа

**Модификатор readonly** запрещает экземплярам структуры изменяться после инициализации. Все поля должны быть отмечены readonly, свойства не должны имет setter (могут иметь init, начиная с C# 9.0).

In [27]:
public readonly struct ReadonlyStruct
{
    public readonly int count;
    
    // Ошибка: должно быть readonly
    public double length;
    
    // Ошибка: set недопустим
    public string Name { get; set; }
}

Unhandled Exception: (6,19): error CS8340: Поля экземпляров в структурах только для чтения должны быть доступны только для чтения.
(9,19): error CS8341: Автоматически реализуемые свойства экземпляра в структурах только для чтения должны быть доступны только для чтения.

Причины пользоваться неизменяемыми типами данных:
- Потокобезопасность
- Надёжность
- Инкапсуляция
- Простота

## 10. Nullable

Структуры можно делать Nullable, т.е. добавить поддержку значения null. 

In [28]:
Nullable<int> nullableIntValue = 3;
nullableIntValue

3

In [29]:
// Упрощённый синтаксис того же самого
int? nullableIntNull = null;
nullableIntNull

Для проверки существования значения используется свойство HasValue, для получения значения - Value.

In [30]:
nullableIntValue.HasValue

True

In [31]:
nullableIntValue.Value

3

In [32]:
nullableIntNull.HasValue

False

Если попробуете достать значение которого нет - получите исключение.

In [33]:
nullableIntNull.Value

Unhandled Exception: Nullable object must have a value.

**Nullable<T> это всё ещё структура!**

## 11. Общие рекомендации

- Ввиду особенностей структур, не рекомендуется создавать структуры больших размеров (>16 байт).
- Следует помнить о дорогих операциях упаковки-распаковки, особенно при реализации интерфейсов.
- Обычно структуры используют для определения легковесных типов без сложного состояния и поведения. Хорошие примеры - координаты на плоскости или рациональное число.
- Желательно делать их immutable (неизменяемыми). Для этого есть модификатор readonly.